In [3]:
#from src.retriever import Retriever
import os,subprocess
import sys
sys.path.append('/root/car_sales_QA/src')

from FlagEmbedding import BGEM3FlagModel,FlagReranker
from dotenv import load_dotenv
from LLM_utils import LLM
from retriever import Retriever
from utils import set_proxy
load_dotenv()
%load_ext autotime

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

#unset http_proxy && unset https_proxy

time: 284 µs (started: 2025-02-19 19:51:32 +08:00)


In [4]:
retriever = Retriever(embedder = 'BAAI/bge-m3',reranker = 'BAAI/bge-reranker-v2-m3',collection_name = 'sales_qa')

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

time: 5.45 s (started: 2025-02-19 19:51:38 +08:00)


In [ ]:

model_id = 'ep-20250213200344-crq6r'
#model_id = 'LLM-Research/Meta-Llama-3.1-8B-Instruct-AWQ-INT4'
gen_llm = LLM(model_id,
              api_key = os.getenv('ARK_API_KEY'),base_url = os.getenv('ARK_BASE_URL')
             )
#model_id = 'LLM-Research/Meta-Llama-3.1-8B-Instruct-AWQ-INT4'
gen_llm('hi')

In [ ]:
import os
import subprocess
from contextlib import contextmanager

@contextmanager
def set_proxy():
    
    # Run the bash command to source the network configuration and capture the proxy settings
    result = subprocess.run(
        'bash -c "source /etc/network_turbo && env | grep proxy"', 
        shell=True, capture_output=True, text=True
    )

    # Set the proxy environment variables from the command output
    proxies = {}
    for line in result.stdout.splitlines():
        if '=' in line:
            var, value = line.split('=', 1)
            os.environ[var] = value
            
    # Yield control back to the block of code using the context manager
    yield

    # Restore the original proxy environment variables
    for key, value in os.environ.items():
        if 'proxy' in key:
            os.environ.pop(key, None)  # Remove the key if it wasn't set originally
with set_proxy():
    print(1)
proxies = {key: os.environ.get(key) for key in os.environ if 'proxy' in key.lower()}
print(proxies)

In [ ]:
import os,subprocess
import sys
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value


In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history: list):
        return "", history + [{"role": "user", "content": user_message}]
        
    def bot(history: list):
        bot_message = random.choice([""""Hey! 😊 I’m here to make your car-buying experience smooth and stress-free. Tell me a bit about what you’re looking for, and I’ll get you started!""",
                                    """Hello and welcome! 🎉 Ready to find your dream car? Let me guide you through our latest models, special offers, and financing options. What are you looking for?""",
                                    """"Hey there! 👋 Excited to help you find the perfect car today! Whether it’s something sleek, spacious, or sporty, I’ve got you covered. What kind of vehicle are you dreaming of?"""])
        
        history.append({"role": "assistant", "content": ""})
        for character in bot_message:
            history[-1]['content'] += character
            time.sleep(0.05)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch()

In [ ]:
import os, sys
sys.path.append('/root/data_disk/src')

from retriever import Retriever
from LLM_utils import LLM
from prompt import generate_prompt